<a href="https://colab.research.google.com/github/cnn22/SingerSongwriter/blob/main/Emotions_Classification_Model_Version_1_Finalized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
%cd .
! git clone https://github.com/surmenok/keras_lr_finder
%cd keras_lr_finder
! python3 -m pip install --editable .
%cd /content

/content
Cloning into 'keras_lr_finder'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 68 (delta 1), reused 0 (delta 0), pack-reused 62
Unpacking objects: 100% (68/68), done.
/content/keras_lr_finder
Obtaining file:///content/keras_lr_finder
  Running setup.py develop for keras-lr-finder
/content


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
#from keras_lr_finder import LRFinder

# Reading and Cleaning data

In [ ]:
#Read in data
train_df = pd.read_csv('train.txt', ';', header=None, names=['Sentence', 'Emotion'], index_col=False)
test_df = pd.read_csv('test.txt', ';', header=None, names=['Sentence', 'Emotion'], index_col=False)

In [ ]:
test_df.Emotion.unique()

array(['sadness', 'joy', 'fear', 'anger', 'love', 'surprise'],
      dtype=object)

## Turn into number target labels

In [ ]:
train_df.loc[train_df.Emotion == 'sadness', 'Emotion'] = 0
train_df.loc[train_df.Emotion == 'anger', 'Emotion'] = 1
train_df.loc[train_df.Emotion == 'love', 'Emotion'] = 2
train_df.loc[train_df.Emotion == 'surprise', 'Emotion'] = 3
train_df.loc[train_df.Emotion == 'fear', 'Emotion'] = 4
train_df.loc[train_df.Emotion == 'joy', 'Emotion'] = 5

In [ ]:
test_df.loc[test_df.Emotion == 'sadness', 'Emotion'] = 0
test_df.loc[test_df.Emotion == 'anger', 'Emotion'] = 1
test_df.loc[test_df.Emotion == 'love', 'Emotion'] = 2
test_df.loc[test_df.Emotion == 'surprise', 'Emotion'] = 3
test_df.loc[test_df.Emotion == 'fear', 'Emotion'] = 4
test_df.loc[test_df.Emotion == 'joy', 'Emotion'] = 5

## Process Data for TensorFlow

In [ ]:
train_df.Emotion = train_df.Emotion.astype('int64')
test_df.Emotion = test_df.Emotion.astype('int64')

In [ ]:
#converting pandas df into a 'TensorSliceDataset' type so we can use tensor flow objects...

train = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(train_df['Sentence'].values, tf.string),
            tf.cast(train_df['Emotion'].values, tf.int64)
        )
    )
)

test = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(test_df['Sentence'].values, tf.string),
            tf.cast(test_df['Emotion'].values, tf.int64)
        )
    )
)

## Split Data into Test and Train

In [ ]:
#Shuffle the train and test data sets
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train = train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test = test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for example, label in train.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'im not feeling real strong lately'
 b'i know that i shouldn t let people decide my happiness but damn it feels like i either have to risk my happiness to please other people that s how much i hate this school this school is fucking pathetic and doesn t deserve my time and money'
 b'i still feel very amorous']

labels:  [5 0 2]


# Encode Text


In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)

encoder.adapt(train.map(lambda text, label: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'i', 'feel', 'and', 'to', 'the', 'a', 'that',
       'feeling', 'of', 'my', 'in', 'it', 'like', 'so', 'for', 'im', 'me',
       'but'], dtype='<U13')

# Import and Load Weights into New Model Instance

In [1]:
#creates a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=32,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dropout(0.65),
      tf.keras.layers.Dense(6) #this is the output layer, since we have 6 emotions we should have 6 as the output number instead of 1
    ])


  model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics=['accuracy'])

  return model

In [ ]:
new_model = create_model()
new_model.load_weights('saved-weights/emotion-weights-v1')
new_model.summary()

In [ ]:
loss, acc = model.evaluate(test)

# Make predictions!

In [ ]:
#sadness', 'joy', 'fear', 'anger', 'love', 'surprise'
sample_text = """I got my driver's license last week
Just like we always talked about
'Cause you were so excited for me
To finally drive up to your house
But today I drove through the suburbs
Crying 'cause you weren't around

And you're probably with that blonde girl
Who always made me doubt
She's so much older than me
She's everything I'm insecure about
Yeah, today I drove through the suburbs
'Cause how could I ever love someone else?

And I know we weren't perfect but I've never felt this way for no one
And I just can't imagine how you could be so okay now that I'm gone
Guess you didn't mean what you wrote in that song about me
'Cause you said forever now I drive alone past your street

And all my friends are tired
Of hearing how much I miss you but
I kinda feel sorry for them
'Cause they'll never know you the way that I do
Yeah, today I drove through the suburbs
And pictured I was driving home to you

And I know we weren't perfect but I've never felt this way for no one
Oh, and I just can't imagine how you could be so okay now that I'm gone
I guess you didn't mean what you wrote in that song about me
'Cause you said forever now I drive alone past your street

Red lights
Stop signs
I still see your face
In the white cars
Front yards
Can't drive past the places
We used to
Go to
'Cause I still fucking love you, babe

Sidewalks
We crossed
I still hear your voice
In the traffic
We're laughing
Over all the noise
God, I'm so blue
Know we're through
But I still fucking love you, babe

I know we weren't perfect but I've never felt this way for no one
And I just can't imagine how you could be so okay now that I'm gone
Guess you didn't mean what you wrote in that song about me
'Cause you said forever now I drive alone past your street
Yeah, you said forever now I drive alone past your street
"""        
predictions = new_model.predict(np.array([sample_text]))
print(predictions)
#return position of max
MaxPosition=np.argmax(predictions) 
prediction_label=['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'][MaxPosition]
print(prediction_label) 